In [1]:
#@title
import os
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [12]:
############################ 加载数据 ############################
df_all = pd.read_csv("drive/My Drive/sougou/data/all_data.csv",encoding="UTF-8",index_col=0)
df_all['Age'] = df_all['Age'] - 1
df_all['Gender'] = df_all['Gender'] - 1
df_all['Education'] = df_all['Education'] - 1
df_all.head()

,ID,Age,Gender,Education,Query_List,Query_List1,jieba,jieba2
0,22DD920316420BE2DF8D6EE651BA174B,0.0,0.0,3.0,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...,柔和双沟\t女生\t中财网首页 财经\t网址\t周公解梦大全查询2345\t曹云金再讽郭德纲...,柔和 双沟 女生 中财网 首页 财经 网址 周公 解梦 大全 查询 2345 曹云金 再讽 ...,柔和 双沟 女生 中财网 首页 财经 网址 周公 解梦 大全 查询 2345 曹云金 再...
1,43CC3AF5A8D6430A3B572337A889AFE4,1.0,0.0,2.0,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只...","广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只...",广州 厨宝 烤箱 世情 人情 雨送 黄昏 花易落 风干 泪痕 厦门 酒店用品 批发市场 不想...,"广州 厨宝 烤箱 世情 薄 , 人情 恶 , 雨送 黄昏 花易落 , 晓 风干 , 泪痕 厦..."
2,6931EFC26D229CCFCEA125D3F3C21E57,3.0,1.0,2.0,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...,受欢迎 狗狗 排行榜 场景 范例 三维 绘图 软件 酸奶 壮观 衣服 网站 动漫 绘图 软件...,最 受欢迎 狗狗 排行榜 舶 怎么 读 场景 描 写 范例 三维 绘图 软件 枣 和 酸...
3,E780470C3BB0D340334BD08CDCC3C71A,1.0,1.0,3.0,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...,干槽症 自愈 太太 万岁 舒心 美国 干槽症 眼皮 怎么回事 麦当劳 旋风 勺子 吉林市 鹿...,干槽症 能 自愈 吗 太太 万岁 叶 舒心 去 没 去 美国 干槽症 右 眼皮 下面 一直 ...
4,0D58894CD481D4B3039000661665C0DB,2.0,1.0,2.0,貔貅饲育指南\t韩姨\t贵阳学古琴\t洛基\t风速狗\t综英美]店主不好吃\t>重生之被拐儿...,貔貅饲育指南\t韩姨\t贵阳学古琴\t洛基\t风速狗\t综英美]店主不好吃\t>重生之被拐儿...,貔貅 饲育 指南 韩姨 贵阳 古琴 洛基 风速 英美 店主 好吃 重生 儿童 寻家记 锦里 ...,貔貅 饲育 指南 韩姨 贵阳 学 古琴 洛基 风速 狗 综 英美 ] 店主 不 好吃 > 重...


In [0]:
df1 = pd.read_csv("drive/My Drive/sougou/data/statistical.csv",encoding="UTF-8")
df2 = pd.read_csv("drive/My Drive/sougou/data/lr_prob_gender.csv",encoding="UTF-8")
df3 = pd.read_csv("drive/My Drive/sougou/data/lr_prob_age.csv",encoding="UTF-8")
df4 = pd.read_csv("drive/My Drive/sougou/data/lr_prob_education.csv",encoding="UTF-8")
df5 = pd.read_csv("drive/My Drive/sougou/data/bnb_prob_gender.csv",encoding="UTF-8")
df6 = pd.read_csv("drive/My Drive/sougou/data/bnb_prob_age.csv",encoding="UTF-8")
df7 = pd.read_csv("drive/My Drive/sougou/data/bnb_prob_education.csv",encoding="UTF-8")
df8 = pd.read_csv("drive/My Drive/sougou/data/mnb_prob_gender.csv",encoding="UTF-8")
df9 = pd.read_csv("drive/My Drive/sougou/data/mnb_prob_age.csv",encoding="UTF-8")
df10 = pd.read_csv("drive/My Drive/sougou/data/mnb_prob_education.csv",encoding="UTF-8")
df11 = pd.read_csv("drive/My Drive/sougou/data/dbowd2v_gender.csv",encoding="UTF-8")
df12 = pd.read_csv("drive/My Drive/sougou/data/dbowd2v_age.csv",encoding="UTF-8")
df13 = pd.read_csv("drive/My Drive/sougou/data/dbowd2v_education.csv",encoding="UTF-8")
df14 = pd.read_csv("drive/My Drive/sougou/data/dmd2v_gender.csv",encoding="UTF-8")
df15 = pd.read_csv("drive/My Drive/sougou/data/dmd2v_age.csv",encoding="UTF-8")
df16 = pd.read_csv("drive/My Drive/sougou/data/dmd2v_education.csv",encoding="UTF-8")
df17 = pd.read_csv("drive/My Drive/sougou/data/w2v_avg.csv",encoding="UTF-8")

In [14]:
df = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17], axis=1)
print(df.columns)

Index(['Query_num', 'Query_len', 'Ave_query_len', 'Max_query_List1',
       'Space_num', 'Url_num', 'Man_num', 'Woman_num', 'tfidf_lr_1',
       'tfidf_lr_2',
       ...
       'w2v_avg_290', 'w2v_avg_291', 'w2v_avg_292', 'w2v_avg_293',
       'w2v_avg_294', 'w2v_avg_295', 'w2v_avg_296', 'w2v_avg_297',
       'w2v_avg_298', 'w2v_avg_299'],
      dtype='object', length=378)


In [0]:
def xgb_acc_score(preds,dtrain):
    y_true = dtrain.get_label()
    y_pred = np.argmax(preds,axis=1)
    return [('acc',np.mean(y_true == y_pred))]

In [29]:
from sklearn.model_selection import train_test_split
############################ Gender ############################
tr_num = 88492
x = df.iloc[:tr_num]
y = df_all["Gender"][:tr_num]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=2020,stratify=y)

num_class = 2
seed = 10
esr = 25
evals = 1
n_trees = 1000
ss = 0.5
mc = 0.8
md = 7
gm = 1

params = {
    "objective": "multi:softprob",
    "booster": "gbtree",
#     "eval_metric": "merror",
    "num_class":num_class,
    'max_depth':md,
    'min_child_weight':mc,
    'subsample':ss,
    'colsample_bytree':0.8,
    'gamma':gm,
    "eta": 0.01,
    "lambda":0,
    'alpha':0,
    "silent": 1,
    "seed":seed,
}
dtrain = xgb.DMatrix(x_train, y_train)
dvalid = xgb.DMatrix(x_test, y_test)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
bst = xgb.train(params, dtrain, n_trees, evals=watchlist,feval=xgb_acc_score,maximize=True,
                early_stopping_rounds=esr, verbose_eval=evals)

[0]	train-merror:0.128196	eval-merror:0.152441	train-acc:0.871804	eval-acc:0.847559
Multiple eval metrics have been passed: 'eval-acc' will be used for early stopping.

Will train until eval-acc hasn't improved in 25 rounds.
[1]	train-merror:0.127518	eval-merror:0.145661	train-acc:0.872482	eval-acc:0.854339
[2]	train-merror:0.127099	eval-merror:0.145322	train-acc:0.872901	eval-acc:0.854678
[3]	train-merror:0.126711	eval-merror:0.144681	train-acc:0.873289	eval-acc:0.855319
[4]	train-merror:0.12592	eval-merror:0.14438	train-acc:0.87408	eval-acc:0.85562
[5]	train-merror:0.12542	eval-merror:0.143589	train-acc:0.87458	eval-acc:0.856411
[6]	train-merror:0.125888	eval-merror:0.143928	train-acc:0.874112	eval-acc:0.856072
[7]	train-merror:0.125468	eval-merror:0.143589	train-acc:0.874532	eval-acc:0.856411
[8]	train-merror:0.125597	eval-merror:0.144116	train-acc:0.874403	eval-acc:0.855884
[9]	train-merror:0.125452	eval-merror:0.143589	train-acc:0.874548	eval-acc:0.856411
[10]	train-merror:0.12585

In [30]:
############################ Age ############################
tr_num = 88492
x = df.iloc[:tr_num]
y = df_all["Age"][:tr_num]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=2020,stratify=y)

num_class = 6
seed = 10
esr = 25
evals = 1
n_trees = 1000
ss = 0.5
mc = 3
md = 7
gm = 2

params = {
    "objective": "multi:softprob",
    "booster": "gbtree",
#     "eval_metric": "merror",
    "num_class":num_class,
    'max_depth':md,
    'min_child_weight':mc,
    'subsample':ss,
    'colsample_bytree':0.8,
    'gamma':gm,
    "eta": 0.01,
    "lambda":0,
    'alpha':0,
    "silent": 1,
    "seed":seed,
}
dtrain = xgb.DMatrix(x_train, y_train)
dvalid = xgb.DMatrix(x_test, y_test)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
bst = xgb.train(params, dtrain, n_trees, evals=watchlist,feval=xgb_acc_score,maximize=True,
                early_stopping_rounds=esr, verbose_eval=evals)

[0]	train-merror:0.34641	eval-merror:0.375697	train-acc:0.65359	eval-acc:0.624303
Multiple eval metrics have been passed: 'eval-acc' will be used for early stopping.

Will train until eval-acc hasn't improved in 25 rounds.
[1]	train-merror:0.338758	eval-merror:0.366958	train-acc:0.661242	eval-acc:0.633042
[2]	train-merror:0.334027	eval-merror:0.364811	train-acc:0.665972	eval-acc:0.635189
[3]	train-merror:0.332623	eval-merror:0.364058	train-acc:0.667377	eval-acc:0.635942
[4]	train-merror:0.329911	eval-merror:0.362137	train-acc:0.670089	eval-acc:0.637863
[5]	train-merror:0.329265	eval-merror:0.361496	train-acc:0.670735	eval-acc:0.638504
[6]	train-merror:0.328862	eval-merror:0.36014	train-acc:0.671138	eval-acc:0.63986
[7]	train-merror:0.32694	eval-merror:0.360328	train-acc:0.67306	eval-acc:0.639672
[8]	train-merror:0.326569	eval-merror:0.359801	train-acc:0.673431	eval-acc:0.640199
[9]	train-merror:0.326214	eval-merror:0.360291	train-acc:0.673786	eval-acc:0.639709
[10]	train-merror:0.32600

In [31]:
############################ Education ############################
tr_num = 88492
x = df.iloc[:tr_num]
y = df_all["Education"][:tr_num]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=2020,stratify=y)

num_class = 6
seed = 10
esr = 25
evals = 1
n_trees = 1000
ss = 0.9
mc = 2
md = 8
gm = 2

params = {
    "objective": "multi:softprob",
    "booster": "gbtree",
#     "eval_metric": "merror",
    "num_class":num_class,
    'max_depth':md,
    'min_child_weight':mc,
    'subsample':ss,
    'colsample_bytree':0.8,
    'gamma':gm,
    "eta": 0.01,
    "lambda":0,
    'alpha':0,
    "silent": 1,
    "seed":seed,
}
dtrain = xgb.DMatrix(x_train, y_train)
dvalid = xgb.DMatrix(x_test, y_test)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
bst = xgb.train(params, dtrain, n_trees, evals=watchlist,feval=xgb_acc_score,maximize=True,
                early_stopping_rounds=esr, verbose_eval=evals)

[0]	train-merror:0.310361	eval-merror:0.361157	train-acc:0.689639	eval-acc:0.638843
Multiple eval metrics have been passed: 'eval-acc' will be used for early stopping.

Will train until eval-acc hasn't improved in 25 rounds.
[1]	train-merror:0.302741	eval-merror:0.35061	train-acc:0.697259	eval-acc:0.64939
[2]	train-merror:0.300578	eval-merror:0.346768	train-acc:0.699422	eval-acc:0.653232
[3]	train-merror:0.298463	eval-merror:0.344207	train-acc:0.701537	eval-acc:0.655793
[4]	train-merror:0.29701	eval-merror:0.344508	train-acc:0.70299	eval-acc:0.655492
[5]	train-merror:0.295606	eval-merror:0.344207	train-acc:0.704394	eval-acc:0.655793
[6]	train-merror:0.295331	eval-merror:0.344169	train-acc:0.704669	eval-acc:0.655831
[7]	train-merror:0.294104	eval-merror:0.34319	train-acc:0.705896	eval-acc:0.65681
[8]	train-merror:0.294282	eval-merror:0.34383	train-acc:0.705718	eval-acc:0.65617
[9]	train-merror:0.29383	eval-merror:0.343378	train-acc:0.70617	eval-acc:0.656622
[10]	train-merror:0.293265	ev